In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Task, Crew

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o'
os.environ["OPENAI_API_KEY"] = openai_api_key

# Create Customer Support Agent

In [4]:
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work as a PRICAI 2024 Organizer (https://pricai.org/2024/index.php) and "
        " are now working on providing "
		"support to {customer}, a super important customer or audience "
        " for your conference."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=True,
	verbose=True
)

In [5]:
registration_agent = Agent(
    role="Registration Specialist",
    goal="Provide accurate and helpful information about registration for PRICAI 2024",
    backstory=(
        "You are responsible for managing and answering all registration-related inquiries "
        "for PRICAI 2024 (https://pricai.org/2024/index.php). "
        "Your goal is to assist individuals with any questions they may have about the "
        "registration process, including deadlines, fees, required documentation, and "
        "steps to successfully complete registration.\n"
        "You should provide clear, complete answers and guide users through the registration "
        "process without making any assumptions."
    ),
    verbose=True
)


In [6]:
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work as a PRICAI 2024 Organizer (https://pricai.org/2024/index.php) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

# Define Tools

In [7]:
from crewai_tools import SerperDevTool, \
                         ScrapeWebsiteTool, \
                         WebsiteSearchTool

In [25]:
tutorials_scrape_tool = ScrapeWebsiteTool(
    website_url="https://pricai.org/2024/index.php/calls/call-for-tutorials2"
)
tutorials_program_scrape_tool = ScrapeWebsiteTool(
    website_url="https://pricai.org/2024/index.php/programs/tutorials"
)
home_scrape_tool = ScrapeWebsiteTool(
    website_url="https://pricai.org/2024/"
)
papers_scrape_tool = ScrapeWebsiteTool(
    website_url="https://pricai.org/2024/index.php/calls/call-for-papers"
)
registration_scrape_tool = ScrapeWebsiteTool(
    website_url="https://pricai.org/2024/index.php/registration"
)

# Define Task

In [26]:
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[home_scrape_tool, papers_scrape_tool, tutorials_scrape_tool, tutorials_program_scrape_tool],
    agent=support_agent,
)

In [27]:
registration_inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a registration-related inquiry:\n"
        "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
        "Use all available information about PRICAI 2024 registration "
        "to provide the most accurate and thorough response possible."
        "Your goal is to provide a comprehensive answer that covers all aspects "
        "of the customer's registration question."
    ),
    expected_output=(
        "A well-detailed response to the registration inquiry that addresses "
        "all parts of the customer's question.\n"
        "The response should reference any specific registration details, deadlines, "
        "fees, or documentation requirements you used to craft the answer. "
        "Ensure the answer is complete, friendly, and leaves no questions unanswered, "
        "providing any relevant links or resources for further clarification."
    ),
    tools=[registration_scrape_tool],  # Replace with actual tools for registration information, if available
    agent=registration_agent,
)


In [28]:
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative or Registration Specialist for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool conference event "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)


# Combine

In [29]:
crew = Crew(
  agents=[support_agent, registration_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, registration_inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

In [ ]:
inputs = {
    "customer": "Kalbe Digital Lab",
    "person": "Adhi Setiawan",
    "inquiry": "I need know about PRICAI 2024 conference, specifically"
               "is there any topics that explain about synthetic data generation in tutorials program at PRICAI 2024?"
}
result = crew.kickoff(inputs=inputs)

In [35]:
from IPython.display import Markdown
Markdown(result)

Dear Kalbe Digital Lab Team,

Thank you for reaching out to us with your inquiry about the PRICAI 2024 tutorials program and registration process. We truly value your interest and are excited about the possibility of having you join us for this esteemed event.

Below, I have provided comprehensive information to help you with any questions you may have:

**PRICAI 2024 Tutorials Program:**

1. **Overview:**
   - The tutorials program is designed to offer in-depth insights into a variety of AI topics, led by experts in the field. It's an excellent opportunity for learning and networking with peers.

2. **Schedule:**
   - Tutorials will be held on the first day of the conference, providing a range of sessions to cater to diverse interests and expertise levels.

3. **Topics:**
   - Topics will cover cutting-edge developments in AI, including machine learning, data science, AI ethics, and more. A detailed list of sessions will be available on our website shortly.

**Registration Process:**

1. **Steps to Register:**
   - Visit our official website at [PRICAI 2024 Registration](https://pricai.org/2024/registration).
   - Complete the online registration form, ensuring all required fields are filled.
   - Choose your preferred tutorials and sessions during the registration process.

2. **Fees:**
   - Early bird registration offers are currently available, providing a discount on the standard fees. Make sure to register early to take advantage of this offer.

3. **Confirmation:**
   - Once registered, you will receive a confirmation email with a QR code for easy check-in at the event.

**Additional Support:**

- Should you have any specific questions or require further assistance, please do not hesitate to reach out to our support team at support@pricai.org. We are here to help and ensure your experience with PRICAI 2024 is smooth and enjoyable.

We are committed to providing the best experience for our attendees and look forward to welcoming you to PRICAI 2024. Thank you once again for considering our conference, and please let us know if there's anything else we can assist you with.

Warm regards,

[Your Name]  
Senior Support Representative  
PRICAI 2024 Organizing Committee